In [1]:
# Importació de llibreries necessàries per a processament de text, 
# reducció de dimensionalitat, modelatge i avaluació

import pandas as pd
import re
import nltk
import numpy as np
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

# Descàrrega de recursos de NLTK per a la tokenització i lematització

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Funcions auxiliars per al preprocessament de text

# Eliminació d’emojis del text utilitzant una expressió regular que cobreix els rangs Unicode d’emoticones
def eliminar_emojis(text):
    if isinstance(text, str):
        patron = re.compile("[" 
            "\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF"
            "\U0001F680-\U0001F6FF" "\U0001F700-\U0001F77F"
            "\U0001F780-\U0001F7FF" "\U0001F800-\U0001F8FF"
            "\U0001F900-\U0001F9FF" "\U0001FA00-\U0001FA6F"
            "\U0001FA70-\U0001FAFF" "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251" "]+", flags=re.UNICODE)
        return patron.sub(r'', text)
    return text

# Inicialització del lematitzador de WordNet
lematitzador = WordNetLemmatizer()

# Obté el tipus gramatical (POS) d'una paraula, necessari per a una lematització precisa
def obtenir_pos_tag(paraula):
    etiqueta = nltk.pos_tag([paraula])[0][1][0].upper()
    tipus = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tipus.get(etiqueta, wordnet.NOUN)

# Funció de lematització completa d’un text (tokenització + lematització)
def lematitzar_text(text):
    paraules = nltk.word_tokenize(text)
    paraules_lem = [lematitzador.lemmatize(p, obtenir_pos_tag(p)) for p in paraules]
    return ' '.join(paraules_lem)

# Càrrega i preprocessament del conjunt de dades


url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=['etiqueta', 'missatge'])

# Conversió de les etiquetes textuals a valors binaris: 'ham' → 0, 'spam' → 1
df['etiqueta'] = df['etiqueta'].replace({'ham': 0, 'spam': 1})

# Aplicació del netejat de text: eliminació d’emojis i lematització
df['missatge'] = df['missatge'].apply(eliminar_emojis).apply(lematitzar_text)

# Definició de paraules buides

stopwords = ['a', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
            'is', 'it', 'for', 'with', 'that', 'this', 'as', 'was', 'be',
            'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
            'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not']

# Definició de variables d’entrada i sortida

X = df["missatge"]
y = df["etiqueta"]

# Definició de la llista de components (k) per a reducció SVD

valors_k = [100]  # nombre de components a provar per SVD
resultats = []

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tonie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tonie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tonie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
C:\Users\tonie\AppData\Local\Temp\ipykernel_7036\3754130502.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['etiqueta'] = df['etiqueta'].replace({'ham': 0, 'spam': 1})


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Configuració de la validació creuada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Bucle principal: per cada valor de k, entrenar i avaluar el model
for k in valors_k:
    puntuacions = []
    for train_idx, test_idx in cv.split(X, y):
        # Divisió del text en entrenament i test
        X_train_raw, X_test_raw = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Vectorització TF-IDF ajustada exclusivament sobre l'entrenament
        tfidf = TfidfVectorizer(max_features=10000, stop_words=stopwords)
        X_train_tfidf = tfidf.fit_transform(X_train_raw)
        X_test_tfidf = tfidf.transform(X_test_raw)
        # Reducció dimensional SVD ajustada només amb l'entrenament
        svd = TruncatedSVD(n_components=k, random_state=42)
        X_train_svd = svd.fit_transform(X_train_tfidf)
        X_test_svd = svd.transform(X_test_tfidf)
        # Entrenament del model XGBoost
        model = XGBClassifier(
            max_depth=6,
            n_estimators=500,
            random_state=42
        )
        model.fit(X_train_svd, y_train)
        print("iteracio")

        # Càlcul del AUC-ROC sobre el conjunt de test
        probs = model.predict_proba(X_test_svd)[:, 1]
        auc = roc_auc_score(y_test, probs)
        puntuacions.append(auc)

    # Emmagatzemar el resultat mitjà per a aquest valor de k
    resultats.append({
        'k_components': k,
        'auc_roc_mitjana': np.mean(puntuacions)
    })

# Emmagatzamem els resultats
df_resultats = pd.DataFrame(resultats)


iteracio
iteracio
iteracio
iteracio
iteracio


In [ ]:
#  Visualització de les puntuacions AUC obtingudes en les diferents iteracions de validació creuada i veiem resultats

print(puntuacions)
print(df_resultats)

[np.float64(0.9884628670120899), np.float64(0.9848911917098445), np.float64(0.9898042215808326), np.float64(0.9752581980039643), np.float64(0.9909726327502869)]
   k_components  auc_roc_mitjana
0           100         0.985878
